# <font color='blue'>Uni-Facef - Pyspark - Parte 2 </font>

#### Neste notebook vamos fazer algumas transformações e agregações no Dataset

#### Importando os módulos necessários

In [ ]:
# coding: utf-8
import pyspark.sql.functions as sf
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
    .appName('Agregação ReceitasGov') \
    .getOrCreate()

In [ ]:
!pwd

In [ ]:
!ls -l /home/al_morais/Documentos/UniFacef/notebooks/Receitas2020

#### Lendo o "Dataset" Receitas2020 em parquet. 

Repare que estamos fazendo a leitura em uma pasta que pode ter diversos arquivos com a mesma estrutura e que compreende à mesma informação que pode estar chegando no repositório de forma incremental

O "parquet" é um formato tipado e colunar da qual o Spark trabalha com uma performance muito boa. Isso faz com que ele sejam uma excelente opção para trabalhar as "zonas" estruturadas em um Datalake . 

In [ ]:
df_receitas = spark.read.parquet('Receitas2020')

df_receitas.printSchema()

In [ ]:
df_receitas.show(5) 

In [ ]:
df_receitas.select("origem_receita").distinct().show(truncate=False) 

#### O médoto "groupBy()" do PySpark Dataframe retorna o GroupedDataobjeto. Segue  as principais funções contidas nesse objeto:

- count() - Retorna a contagem de linhas para cada grupo.
- mean()  - Retorna a média dos valores de cada grupo.
- max()   - Retorna o máximo de valores para cada grupo.
- min()   - Retorna o mínimo de valores para cada grupo.
- sum()   - Retorna o total de valores para cada grupo.
- avg()   - Retorna a média dos valores de cada grupo.


- agg()   - A função agg() é utilizada para calcular mais de uma valor agregado por vez.

In [ ]:
df_receitas.groupBy("origem_receita") \
    .sum("vr_realizado") \
    .withColumnRenamed("sum(vr_realizado)", "sum_vr_realizado") \
    .show()

#### Crindo um novo Dataframe "df_origem_agg", calculando mais um valor agregado por vez

In [ ]:
df_origem_agg = df_receitas \
    .filter("origem_receita is not null") \
    .groupBy("origem_receita") \
    .agg(sf.sum("vr_realizado").alias("sum_vr_realizado"),
         sf.avg("vr_realizado").alias("avg_vr_realizado"),
         sf.max("vr_realizado").alias("max_vr_realizado")) \
    .withColumn("avg_vr_realizado", sf.round("avg_vr_realizado", 2))

print(df_origem_agg.count())

df_origem_agg.show(truncate=False)

#### Trabalhando com sintaxe SQL no Spark através do método "createOrReplaceTempView()" do objeto Dataframe

- createOrReplaceTempView() - Cria uma nova visualização temporária usando SparkDataFrame na Sessão do Spark. Se já existir uma visão temporária com o mesmo nome, substitui.

In [ ]:
df_receitas.createOrReplaceTempView("v_receitas")

In [ ]:
df_teste = spark.sql(
    """
    select distinct data_lancamento
      from v_receitas
    """)

df_teste.show()

#### Podemos construir querys dimâmicas passando variável e fazendo Interpolação de string

In [ ]:
# Variável
filtro = "origem_receita == 'Contribuicoes' and data_lancamento == '2020-01-21'"

query = f"""
    select *
      from v_receitas
     where {filtro}
"""

print(query)

In [ ]:
spark.sql(query).show(5)

In [ ]:
query_origem_agg = """
    select origem_receita, 
           sum(vr_realizado) as sum_vr_realizado,
           avg(vr_realizado) as avg_vr_realizado,
           max(vr_realizado) as max_vr_realizado
      from v_receitas
     where origem_receita is not null
     group by origem_receita
"""

df_origem_agg2 = spark.sql(query_origem_agg) \
    .withColumn("avg_vr_realizado", sf.round("avg_vr_realizado", 2))

df_origem_agg2.show(truncate=False)

### FIM
###### Documentação: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html